In [1]:
####SUCCESFUL: IMPORTANT

import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat


from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())


cuda:3


In [ ]:
# def true_2D_1(xt): #True function for 2D_1 Klein Gordon Equation x \in [-50,50] , t \in [0,10]
#     y = xt[:,0]*np.cos(xt[:,1])
#     return y.reshape(-1,1)    

In [2]:
#Material Properties This link - https://www.mathworks.com/help/pde/ug/nonlinear-heat-transfer-in-a-thin-plate.html#heatTransferThinPlateExample-1
k = 400
rho = 8960
cp = 386
t_z = 0.01
stef_bolt = 5.670373e-8
hc = 1
Ta = 300
emiss = 0.5


In [3]:
label = "Thinplate_atanh"

x = np.linspace(0,1,250).reshape(-1,1)
y = np.linspace(0,1,250).reshape(-1,1)
#t = np.linspace(0,1,100).reshape(-1,1) #t is actually from 0 to 5000, let us scale it to 0 to 1

X,Y = np.meshgrid(x,y)

X = X.flatten('F').reshape(-1,1)
Y = Y.flatten('F').reshape(-1,1)
 
xy = np.hstack((X,Y))

#initial_pts = np.logical_and(T==0,Y!=0).reshape(-1,)

DBC_pts = (Y == 0).reshape(-1,)


NBC_pts_x0 = (X == 0).reshape(-1,)
NBC_pts_x1 = (X == 1).reshape(-1,)

NBC_pts_y0 = (Y == 0).reshape(-1,)
NBC_pts_y1 = (Y == 1).reshape(-1,)

#xy_initial = xyt[initial_pts,:]
xy_DBC = xy[DBC_pts,:]

xy_NBC_x0 = xy[NBC_pts_x0,:]
xy_NBC_x1 = xy[NBC_pts_x1,:]

#xyt_NBC_y0 = xyt[NBC_pts_y0,:]
xy_NBC_y1 = xy[NBC_pts_y1,:]

#u_initial = np.zeros((np.shape(xy_initial)[0],1))
u_DBC = 1000*np.ones((np.shape(xy_DBC)[0],1))

#xy_I_DBC = np.vstack((xy_initial,xyt_DBC))
#xyt_NBC = np.vstack((xyt_NBC_1,xyt_NBC_2,xyt_NBC_3,xyt_NBC_4))
xy_NBC_x = np.vstack((xy_NBC_x0,xy_NBC_x1))
#xyt_NBC_y = np.vstack((xyt_NBC_y0,xyt_NBC_y1))
xy_NBC_y = np.vstack((xy_NBC_y1))

#u_I_DBC = np.vstack((u_initial,u_DBC))

xy_test_tensor = torch.from_numpy(xy).float().to(device)

lb_xy = xy[0]
ub_xy = xy[-1]

In [4]:
def trainingdata(N_D,N_N,N_f,seed):
    '''Boundary Conditions''' 
    
    np.random.seed(seed)
    
    #choose random N_u points for training
    idx = np.random.choice(xy_DBC.shape[0], N_D, replace=False) 
    xy_D = xy_DBC[idx,:] #choose indices from  set 'idx' (x,t)
    u_D = u_DBC[idx].reshape(-1,1)      #choose corresponding u

    idx = np.random.choice(xy_NBC_x.shape[0], N_D, replace=False) 
    xy_Nx = xy_NBC_x[idx,:] #choose indices from  set 'idx' (x,t)

    idx = np.random.choice(xy_NBC_y.shape[0], N_D, replace=False) 
    xy_Ny = xy_NBC_y[idx,:] #choose indices from  set 'idx' (x,t)

    '''Collocation Points'''
    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    samples = sampling(N_f)
    
    xy_coll = lb_xy + (ub_xy - lb_xy)*samples
    xy_coll = np.vstack((xy_coll, xy_D,xy_Nx,xy_Ny)) # append training points to collocation points 

    return xy_coll, xy_D, u_D, xy_Nx,xy_Ny

In [5]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

     
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        self.iter = 0
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)   
        
        self.beta = Parameter(torch.ones((50,len(layers)-2)))
        self.beta.requiresGrad = True
        
        self.train_loss = []
        self.test_loss = []
        
        self.beta_val = []
        
        self.n = torch.tensor(1.0)
            
    'foward pass'
    def forward(self,xy):
        if torch.is_tensor(xy) != True:         
            xy = torch.from_numpy(xy)                
        
        ubxy = torch.from_numpy(ub_xy).float().to(device)
        lbxy = torch.from_numpy(lb_xy).float().to(device)

                      
        #preprocessing input 
        xy = (xy - lbxy)/(ubxy - lbxy)
        
        #convert to float
        a = xy.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.n*self.activation(self.beta[:,i]*z) 
            
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_D(self,xy_D,u_D):
                
        loss_bc = self.loss_function(self.forward(xy_D), u_D)
                
        return loss_bc
    
    def loss_N(self,xy_Nx,xy_Ny,N_hat):
        
        g1 = xy_Nx.clone()             
        g1.requires_grad = True
        u1 = self.forward(g1)
        
        u1_x_y = autograd.grad(u1,g1,torch.ones([xy_Nx.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        du1_dx = u1_x_y[:,[0]]
        
        g2 = xy_Ny.clone()             
        g2.requires_grad = True
        u2 = self.forward(g2)
        
        u2_x_y = autograd.grad(u2,g2,torch.ones([xy_Ny.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        du2_dy = u2_x_y[:,[1]]
               
        loss_N1 = self.loss_function(du1_dx,N_hat)
        loss_N2 = self.loss_function(du2_dy,N_hat)
        
        #return loss_N1+loss_N2       
        return loss_N1 + loss_N2
    
    def loss_PDE(self, xy_coll, f_hat):
        
        g = xy_coll.clone()             
        g.requires_grad = True
        u = self.forward(g) 
        
        u_x_y = autograd.grad(u,g,torch.ones([xy_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        u_xx_yy = autograd.grad(u_x_y,g,torch.ones(xy_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        #du_dt = u_x_y_t[:,[2]]
        
        d2u_dx2 = u_xx_yy[:,[0]]
        d2u_dy2 = u_xx_yy[:,[1]]    
        

        f = - k*t_z*(d2u_dx2+d2u_dy2) + 2*hc*(u-Ta) + 2*emiss*stef_bolt*(torch.pow(u,4)-Ta**4) 
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xy_D,u_D,xy_Nx,xy_Ny,N_hat,xy_coll,f_hat):

        loss_D = self.loss_D(xy_D,u_D)
        loss_N = self.loss_N(xy_Nx,xy_Ny,N_hat)
        loss_f = self.loss_PDE(xy_coll,f_hat)
        
        loss_val = loss_D + loss_N + loss_f
        
        #print(self.iter,"loss_D:",loss_D.cpu().detach().numpy(),"loss_N:",loss_N.cpu().detach().numpy(),"loss_f:",loss_f.cpu().detach().numpy())
        
        return loss_val
     
    'callable for optimizer'                                       
    def closure(self):
        optimizer.zero_grad()
        
        loss = self.loss(xy_D,u_D,xy_Nx,xy_Ny,N_hat,xy_coll,f_hat)
        self.train_loss.append(loss.cpu().detach().numpy())
        
        u_pred = self.test(xy_test_tensor)
        #self.test_loss.append(np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1)))) #Commented because no true values yet
        self.beta_val.append(self.beta.cpu().detach().numpy())
        
        #print(self.iter,"Train Loss",self.train_loss[-1],"Test Loss",self.test_loss[-1])
        print(self.iter,"Train Loss",self.train_loss[-1])
        
        loss.backward()
        self.iter += 1
  

        return loss        
    'test neural network'
    def test(self,xy_test_tensor):
        u_pred = self.forward(xy_test_tensor)
        u_pred = u_pred.cpu().detach().numpy()
   
        return u_pred

In [6]:
max_reps = 10

train_loss_full = []
test_loss_full = []
beta_full = []
elapsed_time= np.zeros((max_reps,1))

for reps in range(max_reps):
    print(reps)
    
    torch.manual_seed(reps*36)
    N_D = 100 #Total number of data points for 'y'
    N_N = 3500
    N_f = 10000 #Total number of collocation points 
    
    xy_coll_np_array, xy_D_np_array, u_D_np_array,xy_Nx_np_array,xy_Ny_np_array = trainingdata(N_D,N_N,N_f,(reps)*22)
        
    xy_coll = torch.from_numpy(xy_coll_np_array).float().to(device)
    xy_D = torch.from_numpy(xy_D_np_array).float().to(device)
    u_D = torch.from_numpy(u_D_np_array).float().to(device)
    xy_Nx = torch.from_numpy(xy_Nx_np_array).float().to(device)
    xy_Ny = torch.from_numpy(xy_Ny_np_array).float().to(device)
        
    N_hat = torch.zeros(xy_Nx.shape[0],1).to(device)    
    f_hat = torch.zeros(xy_coll.shape[0],1).to(device)

    layers = np.array([2,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
    #layers = np.array([3,100,100,100,100,100,100,100,1])

    PINN = Sequentialmodel(layers)
   
    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())
    
    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=1, 
                              max_iter = 5000, 
                              max_eval = None, 
                              tolerance_grad = -1, 
                              tolerance_change = -1, 
                              history_size = 100, 
                              line_search_fn = 'strong_wolfe')



    start_time = time.time()
        
    optimizer.step(PINN.closure)
    
    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(PINN.train_loss)
    test_loss_full.append(PINN.test_loss)
    elapsed_time[reps] = time.time() - start_time
    beta_full.append(PINN.beta_val)


    print('Training time: %.2f' % (elapsed_time[reps]))
    
mdic = {"train_loss": train_loss_full,"test_loss": test_loss_full,"Time": elapsed_time, "beta": beta_full, "label": label}
savemat(label+'.mat', mdic)

0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0 Train Loss 2122402.5
1 Train Loss 2121532.5
2 Train Loss 2114043.0
3 Train Loss 2090234.0
4 Train Loss 2034210.5
5 Train Loss 2018310.2
6 Train Loss 2010485.0
7 Train Loss 61498536.0
8 Train Loss 3138080.0
9 Train Loss 2019688.5
10 Train Loss 2003334.2
11 Train Loss 1956440.4
12 Train Loss 2065695

303 Train Loss 480202.66
304 Train Loss 480202.66
305 Train Loss 480202.66
306 Train Loss 480202.66
307 Train Loss 1186678700.0
308 Train Loss 480202.66
309 Train Loss 480202.66
310 Train Loss 480202.66
311 Train Loss 480202.66
312 Train Loss 480202.66
313 Train Loss 480202.66
314 Train Loss 1186678700.0
315 Train Loss 480202.66
316 Train Loss 480202.66
317 Train Loss 480202.66
318 Train Loss 480202.66
319 Train Loss 480202.66
320 Train Loss 480202.66
321 Train Loss 1186678700.0
322 Train Loss 480202.66
323 Train Loss 480202.66
324 Train Loss 480202.66
325 Train Loss 480202.66
326 Train Loss 480202.66
327 Train Loss 480202.66
328 Train Loss 1186678700.0
329 Train Loss 480202.66
330 Train Loss 480202.66
331 Train Loss 480202.66
332 Train Loss 480202.66
333 Train Loss 480202.66
334 Train Loss 480202.66
335 Train Loss 1186678700.0
336 Train Loss 480202.66
337 Train Loss 480202.66
338 Train Loss 480202.66
339 Train Loss 480202.66
340 Train Loss 480202.66
341 Train Loss 480202.66
342 Train 

630 Train Loss 480202.66
631 Train Loss 480202.66
632 Train Loss 480202.66
633 Train Loss 480202.66
634 Train Loss 480202.66
635 Train Loss 480202.66
636 Train Loss 1186678700.0
637 Train Loss 480202.66
638 Train Loss 480202.66
639 Train Loss 480202.66
640 Train Loss 480202.66
641 Train Loss 480202.66
642 Train Loss 480202.66
643 Train Loss 1186678700.0
644 Train Loss 480202.66
645 Train Loss 480202.66
646 Train Loss 480202.66
647 Train Loss 480202.66
648 Train Loss 480202.66
649 Train Loss 480202.66
650 Train Loss 1186678700.0
651 Train Loss 480202.66
652 Train Loss 480202.66
653 Train Loss 480202.66
654 Train Loss 480202.66
655 Train Loss 480202.66
656 Train Loss 480202.66
657 Train Loss 1186678700.0
658 Train Loss 480202.66
659 Train Loss 480202.66
660 Train Loss 480202.66
661 Train Loss 480202.66
662 Train Loss 480202.66
663 Train Loss 480202.66
664 Train Loss 1186678700.0
665 Train Loss 480202.66
666 Train Loss 480202.66
667 Train Loss 480202.66
668 Train Loss 480202.66
669 Train 

954 Train Loss 480202.66
955 Train Loss 480202.66
956 Train Loss 480202.66
957 Train Loss 480202.66
958 Train Loss 1186678700.0
959 Train Loss 480202.66
960 Train Loss 480202.66
961 Train Loss 480202.66
962 Train Loss 480202.66
963 Train Loss 480202.66
964 Train Loss 480202.66
965 Train Loss 1186678700.0
966 Train Loss 480202.66
967 Train Loss 480202.66
968 Train Loss 480202.66
969 Train Loss 480202.66
970 Train Loss 480202.66
971 Train Loss 480202.66
972 Train Loss 1186678700.0
973 Train Loss 480202.66
974 Train Loss 480202.66
975 Train Loss 480202.66
976 Train Loss 480202.66
977 Train Loss 480202.66
978 Train Loss 480202.66
979 Train Loss 1186678700.0
980 Train Loss 480202.66
981 Train Loss 480202.66
982 Train Loss 480202.66
983 Train Loss 480202.66
984 Train Loss 480202.66
985 Train Loss 480202.66
986 Train Loss 1186678700.0
987 Train Loss 480202.66
988 Train Loss 480202.66
989 Train Loss 480202.66
990 Train Loss 480202.66
991 Train Loss 480202.66
992 Train Loss 480202.66
993 Train 

1269 Train Loss 480202.66
1270 Train Loss 480202.66
1271 Train Loss 480202.66
1272 Train Loss 480202.66
1273 Train Loss 1186678700.0
1274 Train Loss 480202.66
1275 Train Loss 480202.66
1276 Train Loss 480202.66
1277 Train Loss 480202.66
1278 Train Loss 480202.66
1279 Train Loss 480202.66
1280 Train Loss 1186678700.0
1281 Train Loss 480202.66
1282 Train Loss 480202.66
1283 Train Loss 480202.66
1284 Train Loss 480202.66
1285 Train Loss 480202.66
1286 Train Loss 480202.66
1287 Train Loss 1186678700.0
1288 Train Loss 480202.66
1289 Train Loss 480202.66
1290 Train Loss 480202.66
1291 Train Loss 480202.66
1292 Train Loss 480202.66
1293 Train Loss 480202.66
1294 Train Loss 1186678700.0
1295 Train Loss 480202.66
1296 Train Loss 480202.66
1297 Train Loss 480202.66
1298 Train Loss 480202.66
1299 Train Loss 480202.66
1300 Train Loss 480202.66
1301 Train Loss 1186678700.0
1302 Train Loss 480202.66
1303 Train Loss 480202.66
1304 Train Loss 480202.66
1305 Train Loss 480202.66
1306 Train Loss 480202.

...

4934 Train Loss nan
4935 Train Loss nan
4936 Train Loss nan
4937 Train Loss nan
4938 Train Loss nan
4939 Train Loss nan
4940 Train Loss nan
4941 Train Loss nan
4942 Train Loss nan
4943 Train Loss nan
4944 Train Loss nan
4945 Train Loss nan
4946 Train Loss nan
4947 Train Loss nan
4948 Train Loss nan
4949 Train Loss nan
4950 Train Loss nan
4951 Train Loss nan
4952 Train Loss nan
4953 Train Loss nan
4954 Train Loss nan
4955 Train Loss nan
4956 Train Loss nan
4957 Train Loss nan
4958 Train Loss nan
4959 Train Loss nan
4960 Train Loss nan
4961 Train Loss nan
4962 Train Loss nan
4963 Train Loss nan
4964 Train Loss nan
4965 Train Loss nan
4966 Train Loss nan
4967 Train Loss nan
4968 Train Loss nan
4969 Train Loss nan
4970 Train Loss nan
4971 Train Loss nan
4972 Train Loss nan
4973 Train Loss nan
4974 Train Loss nan
4975 Train Loss nan
4976 Train Loss nan
4977 Train Loss nan
4978 Train Loss nan
4979 Train Loss nan
4980 Train Loss nan
4981 Train Loss nan
4982 Train Loss nan
4983 Train Loss nan


5345 Train Loss nan
5346 Train Loss nan
5347 Train Loss nan
5348 Train Loss nan
5349 Train Loss nan
5350 Train Loss nan
5351 Train Loss nan
5352 Train Loss nan
5353 Train Loss nan
5354 Train Loss nan
5355 Train Loss nan
5356 Train Loss nan
5357 Train Loss nan
5358 Train Loss nan
5359 Train Loss nan
5360 Train Loss nan
5361 Train Loss nan
5362 Train Loss nan
5363 Train Loss nan
5364 Train Loss nan
5365 Train Loss nan
5366 Train Loss nan
5367 Train Loss nan
5368 Train Loss nan
5369 Train Loss nan
5370 Train Loss nan
5371 Train Loss nan
5372 Train Loss nan
5373 Train Loss nan
5374 Train Loss nan
5375 Train Loss nan
5376 Train Loss nan
5377 Train Loss nan
5378 Train Loss nan
5379 Train Loss nan
5380 Train Loss nan
5381 Train Loss nan
5382 Train Loss nan
5383 Train Loss nan
5384 Train Loss nan
5385 Train Loss nan
5386 Train Loss nan
5387 Train Loss nan
5388 Train Loss nan
5389 Train Loss nan
5390 Train Loss nan
5391 Train Loss nan
5392 Train Loss nan
5393 Train Loss nan
5394 Train Loss nan


5755 Train Loss nan
5756 Train Loss nan
5757 Train Loss nan
5758 Train Loss nan
5759 Train Loss nan
5760 Train Loss nan
5761 Train Loss nan
5762 Train Loss nan
5763 Train Loss nan
5764 Train Loss nan
5765 Train Loss nan
5766 Train Loss nan
5767 Train Loss nan
5768 Train Loss nan
5769 Train Loss nan
5770 Train Loss nan
5771 Train Loss nan
5772 Train Loss nan
5773 Train Loss nan
5774 Train Loss nan
5775 Train Loss nan
5776 Train Loss nan
5777 Train Loss nan
5778 Train Loss nan
5779 Train Loss nan
5780 Train Loss nan
5781 Train Loss nan
5782 Train Loss nan
5783 Train Loss nan
5784 Train Loss nan
5785 Train Loss nan
5786 Train Loss nan
5787 Train Loss nan
5788 Train Loss nan
5789 Train Loss nan
5790 Train Loss nan
5791 Train Loss nan
5792 Train Loss nan
5793 Train Loss nan
5794 Train Loss nan
5795 Train Loss nan
5796 Train Loss nan
5797 Train Loss nan
5798 Train Loss nan
5799 Train Loss nan
5800 Train Loss nan
5801 Train Loss nan
5802 Train Loss nan
5803 Train Loss nan
5804 Train Loss nan


6165 Train Loss nan
6166 Train Loss nan
6167 Train Loss nan
6168 Train Loss nan
6169 Train Loss nan
6170 Train Loss nan
6171 Train Loss nan
6172 Train Loss nan
6173 Train Loss nan
6174 Train Loss nan
6175 Train Loss nan
6176 Train Loss nan
6177 Train Loss nan
6178 Train Loss nan
6179 Train Loss nan
6180 Train Loss nan
6181 Train Loss nan
6182 Train Loss nan
6183 Train Loss nan
6184 Train Loss nan
6185 Train Loss nan
6186 Train Loss nan
6187 Train Loss nan
6188 Train Loss nan
6189 Train Loss nan
6190 Train Loss nan
6191 Train Loss nan
6192 Train Loss nan
6193 Train Loss nan
6194 Train Loss nan
6195 Train Loss nan
6196 Train Loss nan
6197 Train Loss nan
6198 Train Loss nan
6199 Train Loss nan
6200 Train Loss nan
6201 Train Loss nan
6202 Train Loss nan
6203 Train Loss nan
6204 Train Loss nan
6205 Train Loss nan
6206 Train Loss nan
6207 Train Loss nan
6208 Train Loss nan
6209 Train Loss nan
6210 Train Loss nan
6211 Train Loss nan
6212 Train Loss nan
6213 Train Loss nan
6214 Train Loss nan


/home/smartlab/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


In [ ]:
u_pred = PINN.test(xy_test_tensor)
u_pred_3d = u_pred.reshape(250,250)

In [ ]:
fig, ax = plt.subplots(1,1)
cmap = plt.cm.jet

img3 = ax.imshow(np.transpose(u_pred_3d),vmax =1000,vmin=450,cmap = cmap,extent=[0,1,0,1],aspect = 0.75)


In [ ]:
mat = scipy.io.loadmat('Thinplate_steady.mat')


In [ ]:
np.max(u_pred_3d)

In [ ]:
a = 0
for i in range(10):
    a = a + test_loss_full[i][-1]
print(a/10)

In [ ]:
a = 0
for i in range(10):
    a = a + train_loss_full[i][-1]
print(a/10)